In [3]:
# A few packages you may need, uncomment to download via pip !
#!pip install reverse_geocoder
#!pip install wget
#!pip install reverse_geocoder
#!pip install kaleido

In [4]:
#import required packages
import pandas as pd
import numpy as np
import urllib
import requests
import urllib
import wget
import os
import re
import plotly
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import Image
from urllib.request import urlopen
import json
import reverse_geocoder as rg

In [5]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)

<ipython-input-5-4608f95a2a7d>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [6]:
raw_path = os.path.join("data", "raw")
clean_path = os.path.join("data", "clean")

In [7]:
def makeDataDir():
    """
    Creates a data directory with clean and raw subdirectories.
    """
    if not os.path.exists("data"):
        os.makedirs("data")
        print("created data directory")
        
        if not os.path.exists(raw_path):
            os.makedirs(raw_path)
            print(f"created {raw_path} directory")
        else:
            print(f"{raw_path} directory exists!")
        
        if not os.path.exists(clean_path):
            os.makedirs(clean_path)
            print(f"created {clean_path} directory")     
        else:
            print(f"{clean_path} directory exists!")
    
    else:
        print("data directory exists!")

In [8]:
makeDataDir()

data directory exists!


## Datasets

### 1. Census
Two datasets from the ACS 1-year 2019 estimates:

-  **DP05** - Demographic and Housing Estimates (exported from the DP05 table at county level summary from [data.census.gov](data.census.gov)).
-  **DP03** - Economic Characteristics (exported from the DP03 table at county level summary from [data.census.gov](data.census.gov)).


### 2. Climate (temperature and precipitation)
The daily Global Historical Climatology Network (GHCN) dataset contains global daily climate data (including temperature and precipitation) across 100,000 stations (read more here: https://www.ncdc.noaa.gov/ghcn-daily-description).


### 3. COVID-19 
The Johns Hopkins University Center for Systems Science and Engineering COVID-19 dataset contains global daily COVID-19 metrics (including daily case rate) at the County level for US data (read more here: https://github.com/CSSEGISandData/COVID-19).


### 4. UID Lookup Table
A key for US County names with their respective FIPS codes also found in the Johns Hopkins University Center for Systems Science and Engineering [github repo](https://github.com/CSSEGISandData/COVID-19). 

In [9]:
def censusApiToDf(year_estimate = "acs5", features = False, metadata_only = False, geography = "county:*&in=state:*"):
    """
    Takes parameters to extract data for the ACS 2019 DP03 and DP05 datasets.
    Returns a dataframe of ACS features or variables at the county level.
    
    Allows the following parameters to personalize query:
    
    1. year_estimate - Choose between acs1 and acs5 for 1-year and 5-year estimates.
        1-year estimates has more current, but less data.
        5-year estimates has less current, but more data.
        
    2. features - A list of features to extract from ACS data.
    
    3. metadata_only - If True, returns the metadata of features for a given datatable.
    
    4. geography - Specify US locations to extract. Defaults to extracting all counties.
    """
    
    baseAPI = f"https://api.census.gov/data/2019/acs/{year_estimate}"
    
    
    if metadata_only:
        url = f"{baseAPI}/profile/variables"
        
    elif features:
        features = ",".join(features)
        url = f"{baseAPI}/profile?get=NAME,{features}&for={geography}"
        
    else:
        return print("Error: Must either set variables = True, or pass input features to extract.")

    response = requests.get(url)
    jsonResponse = json.loads(response.text)
    
    raw = pd.DataFrame(data= jsonResponse)
    headers = raw.iloc[0]
    raw.columns = headers
    df = raw.loc[1:,:]
    
    return df

In [10]:
census_metadata = censusApiToDf(metadata_only = True).loc[4:, ["name", "label"]]

In [11]:
census_metadata.sample(frac=1, random_state=8).head()

,name,label
149,DP04_0040E,Estimate!!BEDROOMS!!Total housing units!!1 bedroom
544,DP04_0061E,Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available
888,DP05_0073PE,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican
909,DP02_0025E,Estimate!!MARITAL STATUS!!Males 15 years and over
995,DP02_0110PE,"Percent!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Northern America"


The census metadata files both contain a feature name, and a label description delimited by "!!". 

This can be split in order to better visualize what each feature name refers to and select feature names based on each column.

In [12]:
def splitCensusMetadata(df):
    """
    Takes the raw census metdata dataframe and splits the name column by the "!!" delimiter
    """
    split = df.join(df['label'].str.split("!!", expand=True))
    return split

In [13]:
splitCensusMetadata(census_metadata).sample(frac=1, random_state=8).head()

,name,label,0,1,2,3,4,5,6
149,DP04_0040E,Estimate!!BEDROOMS!!Total housing units!!1 bedroom,Estimate,BEDROOMS,Total housing units,1 bedroom,None,None,None
544,DP04_0061E,Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available,Estimate,VEHICLES AVAILABLE,Occupied housing units,3 or more vehicles available,None,None,None
888,DP05_0073PE,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican,Percent,HISPANIC OR LATINO AND RACE,Total population,Hispanic or Latino (of any race),Puerto Rican,None,None
909,DP02_0025E,Estimate!!MARITAL STATUS!!Males 15 years and over,Estimate,MARITAL STATUS,Males 15 years and over,None,None,None,None
995,DP02_0110PE,"Percent!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Northern America",Percent,WORLD REGION OF BIRTH OF FOREIGN BORN,"Foreign-born population, excluding population born at sea",Northern America,None,None,None


The first field in the label is the type of measurement being captured, **Percent** or **Estimate**.
- For race/ethinicity characteristics of each county, we want **Percent**.
- For economic characteristics of each county we want **Percent** insured, and covered by health insurance, as well as the **Estimate** of median household income.

Unfortunately, the delimited id fields are of unequal lengths making it difficult to query unique columns of interest across the rows. 
Instead of filtering by expanded columns, it may be simpler to simply filter on string patterns within the single column.

In [14]:
def findPatterns(df, col, patterns):
    """
    Takes a dataframe, specified columns, and a list of string patterns.
    Returns a dataframe with matches to all strings patterns.
    """

    string_masks = (df[col].str.contains(string, regex = True, case = False) for string in patterns)
    comb_mask = np.vstack(string_masks).all(axis=0)
    final_df = df[comb_mask]

    return final_df

In [15]:
race_ethnicity_cols = findPatterns(df = census_metadata,
                    col = "label",
                    patterns= ["percent!!", 
                    "total", 
                    "one race|hispanic or latino \(of any race\)$", 
                    "black|white|pacific islander$|asian$|hispanic"])

filtered_ethnicity_cols = race_ethnicity_cols[~race_ethnicity_cols["label"].str.contains("Other Asian|!!Other Pacific Islander")]
DP05_cols = filtered_ethnicity_cols["name"].values
filtered_ethnicity_cols

<ipython-input-14-181e6f94199e>:8: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  comb_mask = np.vstack(string_masks).all(axis=0)


,name,label
360,DP05_0052PE,Percent!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander
493,DP05_0044PE,Percent!!RACE!!Total population!!One race!!Asian
796,DP05_0037PE,Percent!!RACE!!Total population!!One race!!White
942,DP05_0038PE,Percent!!RACE!!Total population!!One race!!Black or African American
1225,DP05_0071PE,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)


Next we need to grab economic metrics including:

1. Median income
2. Insurance coverage percent
3. Poverty percent

In [16]:
income_pat = ["estimate!!", "income", "median household"]
insurance_pat = ["percent!!", "with health insurance coverage$", "noninstitutionalized population!!"]
poverty_pat = ["percent!!", "below the poverty level", "all families$"]

economic_cols = pd.concat([findPatterns(census_metadata, "label", income_pat),
                          findPatterns(census_metadata, "label", insurance_pat),
                          findPatterns(census_metadata, "label", poverty_pat)])

DP03_cols = economic_cols["name"].values
economic_cols

<ipython-input-14-181e6f94199e>:8: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  comb_mask = np.vstack(string_masks).all(axis=0)


,name,label
159,DP03_0062E,Estimate!!INCOME AND BENEFITS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)
678,DP03_0096PE,Percent!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage
675,DP03_0119PE,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families


With our needed features at hand, we can now use the census API to grab county-level data for our features of interest

In [17]:
census_features_all = list(DP03_cols) + list(DP05_cols)
census_colnames = {"DP05_0037PE": "white_perc",
                    "DP05_0038PE": "black_perc",
                    "DP05_0039PE": "amer_native_perc",
                    "DP05_0044PE": "asian_perc",
                    "DP05_0052PE": "pacific_perc",
                    "DP05_0071PE": "hispanic_perc",
                    "DP03_0062E": "income_med_dollars",
                    "DP03_0096PE": "insurance_perc",
                    "DP03_0119PE": "poverty_perc"}

census_fips = censusApiToDf(year_estimate="acs1", features= census_features_all).rename(columns=census_colnames)
census_fips["FIPS"] = census_fips["state"] + census_fips["county"]
census_fips.drop(columns = ["state", "county"], inplace= True)

census_fips

,NAME,income_med_dollars,insurance_perc,poverty_perc,pacific_perc,asian_perc,white_perc,black_perc,hispanic_perc,FIPS
1,"Jefferson County, Kentucky",59049,94.1,9.8,0.1,3.0,71.3,22.4,5.9,21111
2,"Hennepin County, Minnesota",82369,95.2,5.6,0.1,7.3,71.4,13.1,7.0,27053
3,"Olmsted County, Minnesota",80096,94.7,2.7,0.3,6.3,83.4,6.2,5.2,27109
4,"Scott County, Minnesota",108761,95.4,2.4,0.0,5.2,83.0,4.9,5.3,27139
5,"Faulkner County, Arkansas",57642,90.6,11.4,0.0,1.1,82.6,12.2,4.2,05045
...,...,...,...,...,...,...,...,...,...,...
836,"Kaufman County, Texas",75775,86.6,8.0,0.0,0.6,79.3,14.1,23.3,48257
837,"Bell County, Texas",54560,85.0,10.3,0.7,2.9,62.1,25.3,25.6,48027
838,"Cameron County, Texas",41123,70.0,22.6,0.0,0.7,93.7,0.9,90.0,48061
839,"El Paso County, Texas",48903,77.8,16.0,0.0,1.2,80.4,3.5,82.9,48141


Next, the weather station dataset needs to downloaded, aggregated, and cleaned up.

In [18]:
def downloadStations(out = raw_path):
    if not os.path.exists(os.path.join(out, "ghcnd-stations.txt")):
        url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'
        file = wget.download(url, out = out)
    else:
        print("ghcnd-stations.txt already exists")

def downloadWeather(out = raw_path):
    if not os.path.exists(os.path.join(out, "2020.csv")):
        url = 'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2020.csv.gz'
        wget.download(url, out = out)

        !gzip -d data/raw/2020.csv.gz
    else:
        print("2020.csv already exists")

In [19]:
def get_vals(line):
    ls = line.split(',')
    station = ls[0]
    time = ls[1]
    val = float(ls[3])
    return [station, time, val]

def get_stations(filename = os.path.join(raw_path, 'ghcnd-stations.txt')):
    df = pd.read_csv(filename, '/t', header=None)
    df = df[0].str.split(expand=True)[[0, 1, 2, 3]]
    df.columns = ['Station', 'Latitude', 'Longitude', 'Elevation']
    df_drop = df[df["Station"].str.startswith("US")].reset_index(drop = True)
    return df_drop

def process_year(col='TAVG', path = raw_path):
    tavg = []
    pattern = re.compile(r"^US.*TAVG")
    with open(os.path.join(path, "2020.csv")) as h:
        l = h.readline()
        while l:
            if re.match(pattern, l) is not None:
                v = get_vals(l)
                tavg.append(get_vals(l))
            l = h.readline()
    df_tavg = pd.DataFrame(tavg, columns=['Station', 'Date', col])
    return df_tavg

def mergeStationAggYear(stations, df_tavg):
    merged = df_tavg.merge(stations, on='Station', how='inner')
    
    merged["TAVG"] = merged["TAVG"]/10 + 273

    # Get mean temp and temp standard deviation across year
    agg_df = merged.loc[:,["Station","Latitude", "Longitude", "TAVG"]].groupby(["Station", "Latitude", "Longitude"]).agg(
        temp_mean = ("TAVG", np.mean),
        temp_std = ("TAVG", np.std))

    #Convert temperture mean to celsius for intepretability
    agg_df["temp_mean"] = agg_df["temp_mean"] - 273
    return agg_df

In [20]:
def weatherToDf():
    """
    The full pipeline to:
    1. Download weather station temperature data (US only)
    2. Merge station location to temp data
    3. Aggregate mean temperature, and standard deviation across the year
    """
    downloadStations()
    print("Downloaded weather stations")
    downloadWeather()
    print("Downloaded temperature data.")
    
    stations = get_stations()
    df_tavg = process_year(col='TAVG')
    merged_agg = mergeStationAggYear(stations, df_tavg).reset_index()
    
    return merged_agg

In [21]:
weather_station_agg = weatherToDf()

ghcnd-stations.txt already exists
Downloaded weather stations
2020.csv already exists
Downloaded temperature data.


<ipython-input-19-48efbb0076e0>:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename, '/t', header=None)


In [22]:
weather_station_agg.head()

,Station,Latitude,Longitude,temp_mean,temp_std
0,USC00244558,48.3042,-114.2636,7.069126,8.606210
1,USC00507783,62.0911,-152.7350,-10.887037,5.443052
2,USR0000AALP,33.8417,-109.1222,10.309563,7.994981
3,USR0000AASI,67.4750,-162.2664,-3.070083,12.362611
4,USR0000ABAN,34.1400,-87.3622,16.246721,7.440664


In [23]:
weather_station_agg.describe()

,temp_mean,temp_std
count,2364.000000,2364.000000
mean,8.768886,8.659630
std,6.476482,2.107901
min,-25.146729,1.235255
25%,4.550273,7.618067
50%,7.836749,8.554756
75%,13.283468,9.383560
max,26.613934,20.707320


Our aggregated dataset contains 2,364 rows which represents each unique weather station. 
The weather stations currently have only latitude and longitude. 
In order to join the weather dataset to the census and covid datasets, the FIPS codes. There are two steps needed to transforms these lat/long values to FIPS codes:

1. Convert latitude and longitude to county/state name using reverse geocoder.
2. Use a lookup table to extract FIPS codes from county/state name. 

In [24]:
#Reverse geocode latitutude and longitutes to county and state
#Function to reverse_geocode counties

def addCountyStateFromLatLong(dataframe, lat, long, US_only = False):
    """
    Takes a dataframe with latitude and longitude and adds the county and state.
    
    Inputs:
    1) Dataframe 
    2) Latitude Column
    3) Longitude Column
    
    Returns: 
    1) Dataframe with County and State
    """
    
    #Use lat and long as tuples to extract county and state 
    query = rg.search([tuple(x) for x in dataframe[[lat, long]].values])
    
    #Initialize lists to extract county and state from our query
    state = []
    county = []
    country = []
    
    if US_only:
        for i in np.arange(0,len(query)):
            state.append(query[i]["admin1"])
            county.append(query[i]["admin2"])
            country.append(query[i]["cc"])
    else:
        for i in np.arange(0,len(query)):
            state.append(query[i]["admin1"])
            county.append(query[i]["admin2"])
    
    #Create dataframe of filled lists
    if US_only:
        sc_df = pd.DataFrame({"state": state,
                        "county": county,
                        "country": country})
    else:
        sc_df = pd.DataFrame({"state": state,
                        "county": county})
    
    #assert that the query and station dataframes are the same size
    assert len(dataframe) == len(sc_df), "Row lengths don't match for input and output. Check lat/long values in input."
    
    #Merge the Station with county and state horizontally
    if US_only:
        concat_df = pd.concat([dataframe,
                           sc_df],
                           axis = 1)

        concat_df = concat_df[concat_df["country"] == "US"].drop(columns = "country")
    else:
        concat_df = pd.concat([dataframe,
                           sc_df],
                           axis = 1)
    
    return concat_df

In [25]:
weather_station_state_county = addCountyStateFromLatLong(weather_station_agg, "Latitude", "Longitude", US_only=True)

Loading formatted geocoded file...


In [26]:
weather_station_state_county.head()

,Station,Latitude,Longitude,temp_mean,temp_std,state,county
0,USC00244558,48.3042,-114.2636,7.069126,8.606210,Montana,Flathead County
1,USC00507783,62.0911,-152.7350,-10.887037,5.443052,Alaska,Kenai Peninsula Borough
2,USR0000AALP,33.8417,-109.1222,10.309563,7.994981,Arizona,Apache County
3,USR0000AASI,67.4750,-162.2664,-3.070083,12.362611,Alaska,Northwest Arctic Borough
4,USR0000ABAN,34.1400,-87.3622,16.246721,7.440664,Alabama,Winston County


In [27]:
def fipsIntToString(sr):
    """
    Takes a series of FIPS integers with NA values
    Returns a FIPS string 5 with 0 padding up front
    """
    string = sr.fillna(0).astype("int32").astype("str")
    padding = string.apply(lambda x: "0"*(5 - len(x)) + x)
    return padding

In [28]:
#Helper function to join county and state with FIPS from a lookup table
def add_FIPS(dataframe):
    """
    This function joins FIPS codes to the state and county outputs of reverse_geocoder
    Inputs:
    1. Dataframe with "state" and "county" columns
    2. Path to FIPS lookup table
    """
    #Download FIPS lookuptable with relevant columns
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"
    lookuptable= pd.read_csv(url).loc[:,["FIPS", "Combined_Key"]]
    
    #Add the FIPS codes
    add_FIPS = dataframe.copy()
    add_FIPS["Combined_Key"] = add_FIPS["county"].str.replace(" County", ", ") + add_FIPS["state"] + ", US"
    climate_FIPS = add_FIPS.merge(lookuptable, on = "Combined_Key")
    climate_FIPS.drop(columns = ["state", "county", "Combined_Key"], inplace = True)

    #Convert FIPS to 5 digit string
    climate_FIPS["FIPS"] = fipsIntToString(climate_FIPS["FIPS"])
    return climate_FIPS

Lastly, we want to aggregate all weather stations within each FIPS code.

In [29]:
weather_station_FIPS = add_FIPS(weather_station_state_county)
weather_FIPS_agg =  weather_station_FIPS.groupby("FIPS").agg(temp_mean = ("temp_mean", np.mean),
                                                            temp_std = ("temp_std", np.mean)).reset_index()

In [30]:
print(f"Weather Dataset:\nNumber of Counties: {len(weather_FIPS_agg)}\nColumns = {weather_FIPS_agg.columns.values}")

Weather Dataset:
Number of Counties: 752
Columns = ['FIPS' 'temp_mean' 'temp_std']


After aggregating temperature features by FIPS codes, we are left with 752 observations.
Each of these represent a county with its

1. FIPS code
2. Temperature mean
3. Temperature standard deviation

The last dataset that needs to be prepared is the COVID-19 dataset.

In [31]:
def downloadCOVID(out = raw_path):
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/12-31-2020.csv"
    
    file = url.split("/")[-1]

    if os.path.exists(out):
        if os.path.exists(os.path.join(out, file)):
            print(file, "already exists")
        else:
          wget.download(url=url, out=out)
    else:
        print(f"The directory {out} does not exist")
    

In [32]:
downloadCOVID()

12-31-2020.csv already exists


In [33]:
def covidUsDf(path = raw_path, file = "12-31-2020.csv"):
    """
    Takes a path and a file containing data from the CSSE COVID-19 Daily Reports

    Returns a dataframe with only US data
    """
    raw = pd.read_csv(os.path.join(path, file))
    us = raw[raw["Country_Region"] == "US"]
    return us

In order to compare each county's COVID-19 infection status adjusted for population, the most useful metric is the **Incident Rate** where:

*Incidence Rate = cases per 100,000 persons*

In [34]:
covid_US = covidUsDf().loc[:, ["FIPS", "Incident_Rate"]]
covid_US["FIPS"] = fipsIntToString(covid_US["FIPS"])
covid_US.head()

,FIPS,Incident_Rate
649,01001,7499.686767
650,01003,6092.709892
651,01005,6133.030868
652,01007,8189.693668
653,01009,8025.801543


We now have 3 datasets which can be joined by FIPS codes which represent US counties. The datasets have the following features:

1. **Census (2019 estimates)**

    - American Native Proportion of the Population
    - Asian Proportion of the Population
    - Black Proportion of the Population
    - Hispanic Proportion of the Population
    - White Proportion of the Population
    - Income in median dollar (annual per household)
    - Proportion of the Population with Insurance
    - Proportion of the Population below the Poverty Level

<br>

2. **Climate (2020 aggregate from 1070 weather stations)**

    - Temperature mean
    - Temperature standard deviation

<br> 

3. **COVID-19 (Cumulative Incidence Rate at the end of 2020, 12-31-2020)**

    - Incidence rate



In [35]:
census_weather_covid_merge = census_fips.merge(weather_FIPS_agg, how= "inner", on = "FIPS").merge(covid_US, how= "inner", on = "FIPS")
census_weather_covid_merge

,NAME,income_med_dollars,insurance_perc,poverty_perc,pacific_perc,asian_perc,white_perc,black_perc,hispanic_perc,FIPS,temp_mean,temp_std,Incident_Rate
0,"Jefferson County, Kentucky",59049,94.1,9.8,0.1,3.0,71.3,22.4,5.9,21111,15.382514,8.917145,6625.828000
1,"Hennepin County, Minnesota",82369,95.2,5.6,0.1,7.3,71.4,13.1,7.0,27053,8.887978,11.746417,6815.458157
2,"Olmsted County, Minnesota",80096,94.7,2.7,0.3,6.3,83.4,6.2,5.2,27109,7.541530,11.488897,5880.866494
3,"Sedgwick County, Kansas",59716,89.3,7.7,0.0,4.3,77.8,8.6,15.0,20173,14.396175,9.938393,7639.494460
4,"Allegheny County, Pennsylvania",64871,96.3,6.6,0.0,3.9,79.4,12.9,2.3,42003,11.866667,9.198555,4424.918486
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,"Wichita County, Texas",51749,84.9,7.2,0.2,2.3,81.3,9.6,19.8,48485,17.574863,8.689958,8144.899040
299,"Gregg County, Texas",53793,80.2,13.4,0.0,1.3,74.1,18.4,19.2,48183,18.798634,7.568277,5495.986123
300,"Lubbock County, Texas",55003,86.0,13.4,0.2,2.2,80.9,6.9,36.3,48303,15.357377,9.507474,13484.282076
301,"Cameron County, Texas",41123,70.0,22.6,0.0,0.7,93.7,0.9,90.0,48061,24.348224,5.120266,6977.216817


According to the United States Census Bureau, there are 3,141 county-equivalents in the U.S., but our merged dataset only contains 228 records. 
Which datasets are lacking the most counties?

In [36]:
def fipsGetAll():
    """
    Downloads and returns all unique US FIPs codes from kjhealy's github repo as integers.
    """
    df = pd.read_csv("https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv", 
                     encoding = 'cp1252')
    fips = df.loc[:,["fips"]].rename(columns = {"fips": "FIPS"})
    return fips

In [37]:
all_fips = fipsGetAll()
all_fips["FIPS"] = fipsIntToString(all_fips["FIPS"])

In [38]:
def fipsCoverage(df, allfips):
    """
    Takes a dataframe with some FIPS codes and compares it to another dataframe with all FIPS codes, both of which must have a "FIPS" columns.
    Returns the percent of FIPS codes covered in the first dataframe.
    """
    proportion = int(df.merge(allfips, on = "FIPS").shape[0]/ allfips.shape[0]*100)
    return proportion

In [39]:
print(f"There are {census_fips.shape[0]} counties in the Census dataset with {fipsCoverage(census_fips, all_fips)}% coverage.")
print(f"There are {weather_FIPS_agg.shape[0]} counties in the Climate dataset with {fipsCoverage(weather_FIPS_agg, all_fips)}% coverage.")
print(f"There are {covid_US.shape[0]} counties in the COVID-19 dataset with {fipsCoverage(covid_US, all_fips)}% coverage.")
print(f"There are {census_weather_covid_merge.shape[0]} counties in the merged dataset with {fipsCoverage(census_weather_covid_merge, all_fips)}% coverage.")

There are 840 counties in the Census dataset with 26% coverage.
There are 752 counties in the Climate dataset with 23% coverage.
There are 3274 counties in the COVID-19 dataset with 99% coverage.
There are 303 counties in the merged dataset with 9% coverage.


In order to geographically visualize which US counties are covered in our datasets, we can map the FIPS codes present in each dataset.

In [40]:
# Map of U.S. Counties
def mapCounties(df, title = "US Counties", savefig = False, fig_dir ="Figures"):

    """
    Takes a dataframe with a "FIPS" column.
    Returns a map of the US with the present counties highlighted.
    """    

    with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
        counties = json.load(response)

    fig = px.choropleth(df, 
                        geojson=counties, 
                        locations='FIPS',
                        scope="usa")

    fig.update_layout(title = title,
                      showlegend = False)
    if savefig:
        if "html" in savefig:
            return plotly.offline.plot(fig, filename= os.path.join(fig_dir, savefig))
        else:
            return fig.write_image(os.path.join(fig_dir, savefig))
    else:
        return fig

In [41]:
mapCounties(census_fips, 
title = f'Counties in United States 2019 Census Estimates ({fipsCoverage(census_fips, all_fips)}% coverage)',
savefig= "Counties_2019_ACS1_Census.png")

In [42]:
mapCounties(weather_FIPS_agg, 
title= f"Counties in GHCN weather dataset ({fipsCoverage(weather_FIPS_agg, all_fips)}% coverage)",
savefig= "Counties_2019_Weather.png")

In [43]:
mapCounties(covid_US, 
title= f"Counties in COVID-19 dataset ({fipsCoverage(covid_US, all_fips)}% coverage)",
savefig= "Counties_2019_COVID19.png")

In [44]:
mapCounties(census_weather_covid_merge, 
title= f"Counties in Merged Census, Weather, and COVID Dataset({fipsCoverage(census_weather_covid_merge, all_fips)}% coverage)", 
savefig= "Counties_2019_ACS1_Census_Weather_COVID.png")

From the maps we can see that:

1. The **COVID-19** dataset has the most coverage, with data on nearly all counties.
2. The **Census 2019 estimates** dataset has moderate coverage, with sparse coverage in the Western US.
3. The **Weather** dataset has modertely low coverage, with sparse coverage in the Eastern US.

After performing inner joins on all three datasets, the full dataset is very limited.

The final, merged, census, weather, and covid dataset has only has 303 records which represents only 9% of all US counties.

There are several reasons why we have such little coverage of US counties including:

1. **Weather stations are not distrubted evenly among counties**. 
    - They appear to be most prevalent along the Rocky Mountains in the Western US.
    - They are sparsley distributed in the MidWest and Eastern US.
2. **The census data is missing many counties**
    - The 1-year 2019 ACS census estimates miss many counties in order to present only the most current information:
        - Only including 2019 data
        - Only including counties with populations > 65,000
        - To read more about the tradeoff between 1-year, 3-year, and 5-year estimates, visit https://www.census.gov/programs-surveys/acs/guidance/estimates.html

This **smaller dataset with the most current feature estimates seems the most appropriate for inference** regarding the effect of demographic and weather features on COVID-19 infection rates.

**For predictions, we can instead want to use the larger, but less current 5-year ACS census estimates**.
Since the Climate dataset 13% coverage, we will keep weather out of the predictive modeling features leaving only demographic features to predict COVID-19 infection rates.

In [45]:
# Use the same features extracted for the acs1 census estimates

acs5_census= censusApiToDf(year_estimate="acs5", features= census_features_all).rename(columns=census_colnames)
acs5_census["FIPS"] = acs5_census["state"] + acs5_census["county"]
acs5_census.drop(columns = ["state", "county"], inplace= True)

In [46]:
acs5_covid_merge = acs5_census.merge(covid_US, how= "inner", on = "FIPS")

In [47]:
acs5_covid_merge

,NAME,income_med_dollars,insurance_perc,poverty_perc,pacific_perc,asian_perc,white_perc,black_perc,hispanic_perc,FIPS,Incident_Rate
0,"Fayette County, Illinois",46650,91.8,12.8,0.1,0.5,93.9,4.7,1.9,17051,12485.939258
1,"Logan County, Illinois",57308,95.5,5.6,0.0,0.8,88.5,6.9,3.4,17107,9452.093088
2,"Saline County, Illinois",44090,95.8,17.9,0.2,0.7,92.7,2.6,1.8,17165,7509.258865
3,"Lake County, Illinois",89427,93.2,5.8,0.0,7.7,75.8,6.8,21.7,17097,6934.324908
4,"Massac County, Illinois",47481,94.6,13.6,0.0,0.2,91.1,5.8,2.9,17127,6484.170781
...,...,...,...,...,...,...,...,...,...,...,...
3189,"Crockett County, Tennessee",44717,88.9,13.1,0.0,0.3,79.5,13.0,10.6,47033,11531.974701
3190,"Lake County, Tennessee",35191,88.7,25.6,0.0,0.2,67.5,28.5,2.3,47095,20068.415051
3191,"Knox County, Tennessee",57470,92.2,9.5,0.0,2.2,85.5,8.7,4.3,47093,6942.610559
3192,"Benton County, Washington",69023,92.7,8.6,0.1,2.6,82.1,1.6,21.7,53005,5720.436421


In [48]:
print(f"There are {acs5_census.shape[0]} counties in the ACS 5-year Census dataset with {fipsCoverage(acs5_census, all_fips)}% coverage.")
print(f"There are {acs5_covid_merge.shape[0]} counties in the merged prediction dataset with {fipsCoverage(acs5_census, all_fips)}% coverage.")

There are 3220 counties in the ACS 5-year Census dataset with 99% coverage.
There are 3194 counties in the merged prediction dataset with 99% coverage.


In [49]:
mapCounties(acs5_census, 
title= f"Counties in acs5 dataset ({fipsCoverage(acs5_census, all_fips)}% coverage)",
savefig= "Counties_2019_ACS5.png")

The 5-year estimates contain an incredible 99% coverage! The 3,220 records has sufficient data for training and validation in our predictive modeling.

In [50]:
def write_csv(df, path, filename):
    """
    Takes a dataframe, path, and filename.
    Returns a written csv.
    Checks if the csv is already written, and if the path is correct.
    """
    file = os.path.join(path, filename)

    if os.path.exists(path):
        if os.path.exists(file):
            print(f"The csv {filename} already exists.")
        else:
            pd.DataFrame.to_csv(df, file)
    else:
        print(f"The path {path} does not exist.")

In [51]:
census_weather_covid_inference_name = "census_weather_covid_inference.csv"

write_csv(census_weather_covid_merge, clean_path, census_weather_covid_inference_name)

The csv census_weather_covid_inference.csv already exists.


In [52]:
census_covid_prediction_name = "census_covid_prediction.csv"
write_csv(acs5_covid_merge, clean_path, census_covid_prediction_name)

The csv census_covid_prediction.csv already exists.


In [ ]:
# With our 2 preprocessed datasets, we can now move on to analysis.